<a href="https://colab.research.google.com/github/tykimos/Keras/blob/master/Pytorch_Fashion_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init

import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

batch_size = 256
learning_rate = 0.0002
num_epoch = 10

mnist_train = dset.FashionMNIST("../", train=True, transform=transforms.ToTensor(), target_transform=None, download=True)
mnist_test = dset.FashionMNIST("../", train=False, transform=transforms.ToTensor(), target_transform=None, download=True)

train_loader = torch.utils.data.DataLoader(mnist_train,batch_size=batch_size, shuffle=True,num_workers=2,drop_last=True)
test_loader = torch.utils.data.DataLoader(mnist_test,batch_size=batch_size, shuffle=False,num_workers=2,drop_last=False)

class Linear(nn.Module):
    def __init__(self):
        super(Linear,self).__init__()
        self.layer = nn.Sequential(
            nn.Linear(784,300),
            nn.ReLU(),
            nn.Linear(300,100),
            nn.ReLU(),
            nn.Linear(100,10)
        )       
        
    def forward(self,x):
        out = x.view(-1, 784)
        out = self.layer(out)
        return out

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

model = Linear().to(device)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

loss_arr =[]
for i in range(num_epoch):
    for j,[image,label] in enumerate(train_loader):
        x = image.to(device)
        y_= label.to(device)
        
        optimizer.zero_grad()
        output = model.forward(x)
        loss = loss_func(output,y_)
        loss.backward()
        optimizer.step()
        
        if j % 1000 == 0:
            print(loss)
            loss_arr.append(loss.cpu().detach().numpy())

correct = 0
total = 0

pred_labels = []

with torch.no_grad():
    for image,label in test_loader:
        x = image.to(device)
        y_= label.to(device)
        
        output = model.forward(x)
        _,output_index = torch.max(output,1)

        total += label.size(0)
        correct += (output_index == y_).sum().float()
        
        pred = output_index.to('cpu').numpy()
        pred_labels.extend(pred)

print("Accuracy of Test Data: {}".format(100*correct/total))
np.savetxt('submit.txt', pred_labels, fmt='%d')            

Extracting ../FashionMNIST/raw/train-images-idx3-ubyte.gz to ../FashionMNIST/raw


Extracting ../FashionMNIST/raw/train-labels-idx1-ubyte.gz to ../FashionMNIST/raw


Extracting ../FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ../FashionMNIST/raw


Extracting ../FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ../FashionMNIST/raw
Processing...
Done!





/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


cuda:0
tensor(2.3027, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.5488, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.4699, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.4277, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.4019, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.4393, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.3508, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.3596, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.3662, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.3331, device='cuda:0', grad_fn=<NllLossBackward>)
Accuracy of Test Data: 86.48999786376953
